# CHAPTER 2 금융 데이터 분석을 위한 파이썬 활용법
----------

* <span style="color:red;">자세하게 읽을 필요 없이, 실습 진행하면서 어려움이 생기면 돌아와서 확인하는 부분</span>

## 2.1 날짜와 시간  
--------------------------------

* python, numpy, pandas 에서 지원하는 datetime 설명 

datetime|datetime64
:--------|:--------
python 표준 라이브러리  | numpy 에서 제공
$10^{-6}$초 표현 | $10^{-18}$초 표현

> pandas는 내부적으로 numpy를 포함하고 있기 때문에 datetime64를 사용한다


### 2.1.3 pandas 의 날짜/시간 표현 
--------------------------------

* timestamp : 특정 시점 
    * 특정 시점은 아래와 같은 함수를 써서 str -> datetime64로 바꿀 수 있다.  
    ```python
    pd.to_datetime(['2020-11-24', '2020-11-30'])
    ```

* period : 기간
    * 일 단위로는 아래와 같이 생성
    ```python
    pd.Period('2020-11-24', 'D')
    ```
* timestamp 와 period 는 아래와 같이 표현할 수 있다
    ```python
    p = pd.Period('2020-11-24')
    t = pd.to_datetime('2020-11-24 23:11')
    p.start_time < t < p.end_time
    ```
* 일정 기간 사이의 날짜 생성하기  
    * 주기(freq)를 이용하면 영업일만으로 구성된 기간 생성도 가능하다.
    ```python
    #영업일만 뽑기
    pd.date_range('2020-11-01', '2020-11-30', freq='B')
    pd.period_range('2020-11-01', '2020-11-30', freq='B')
    
    #각 달의 월요일 뽑기
    pd.date_range('2020-11-01', '2020-11-30', freq='W-MON')
    pd.period_range('2020-11-01', '2020-11-30', freq='W-MON')
    ```

> 주식 거래소 사이트에서 휴장일을 크롤링하여, 휴일을 정의한 후 영업일만 뽑기 등으로 응용이 가능해 보인다

## 2.2 금융 데이터 전처리와 분석을 위한 판다스 사용법
--------------------------------

* pandas 의 데이터 표현 구조
    * Series : 1차원
    * DataFrame : 2차원

> DataFrame 에서 row는 index, column은 변수(Series 객체들의 집합)

* 저자의 팁 : read_csv로 데이터를 읽어올 때, Date 칼럼을 timestamp 로 변환하고 index로 잡아주면 좋다
    ```python
    pd.read_csv('test.csv', index_col='Date', parse_dates=['Date'])
    ```

### 2.2.2 결측치와 이상치 다루기
--------------------------------

* 결측치 : NaN (또는 무한값)
    * NaN 확인하기
        * isna()
            ```python
            df.isna()
            df.isna().sum()
            ```
        * isnull() : isna() 와 동일하게 동작한다
        * isin()
            ```python
            df.isin([np.nan])
            df.isin([np.nan]).sum()
            ```
    * NaN 처리하기
        * 제거
        ```python
        df.dropna(axis='rows') # 결측치가 발생했을 경우 행 방향으로 제거(axis=0) 
        df.dropna(axis='columns') # 결측치가 발생했을 경우 열 방향으로 제거(axis=1)
        ```
        * 다른 값 채우기
        ```python
        df.fillna(method='ffill') # 앞 행 값으로 채우기
        df.fillna(method='bfill') # 뒤 행 값으로 채우기
        
        df.fillna(method='ffill', limit=1) # 앞 행 값으로 채우되, 연속된 NaN이 있으면 1번만 사용
        
        df.fillna(df.mean()) # 각 칼럼의 평균값으로 채우기
        df.fillna(df.mean()['C1']) # C1 칼럼의 평균값으로 채우기
        df.fillna(df.mean()['C1':'C2']) # C1, C2 에 대해서만 각 칼럼의 평균값으로 채우기
        
        df['C2'] = np.where(pd.notnull(df['C2']) == True, df['C2'], df['C1']) # C2 값이 없으면 C1값으로 채우기
        ```
        > 날짜 index가 오름차순인 경우, backward fill 기법은 look-ahead bias 문제를 일으킬 수 있어 사용 시 고려해야 한다  
        > &nbsp;<span style="font-size:0.8em;">※ look-ahead bias : 당시에 쉽게 사용할 수 없었던 데이터가 해당 기간의 시뮬레이션에 사용되는 경우</span>
    * NaN과 무한값 동시에 찾아내기
    ```python
    df[df.isin([np.nan, np.inf, -np.inf]).any(1)]
    ```

### 2.2.3 데이터 선택하기 (슬라이싱, 인덱싱, 서브셋 데이터 추출)
--------------------------------

* indexing 으로 위치를 찾고, 해당 정보를 기반으로 slicing 한다
    * 열 단위로 추출하기
    ```python
    df['C1', 'C2'] # 에러 발생 : ('C1', 'C2') 라는 tuple column 명으로 인식함
    df[['C1', 'C2']] # 이와 같이 list 형식으로 전달해야 한다
    ```
    * 행 단위로 추출하기
    ```python
    df[0:3] # 0, 1, 2 행 추출
    df['2019-11-25':'2019-11-28'] # 인덱스 단위로 추출 -> 내부적으로 str을 index의 형식인 Timestamp 로 변환해 준다
    df['2019-Nov-25':'2019-Nov-28'] # 위와 동일
    ```  

* 실제로는 loc, iloc 을 이용해서 추출하는 작업을 많이 한다 → 행, 열 indexing 가능
    * loc : index label 값 기반
    * iloc : 정수값 기반
    ```python
    df.loc['2019-11-25':'2019-11-28', ['C1', 'C2']] # 위의 열/행 단위로 추출하기의 교집합
    df.iloc[0:3, [0, 1]] # 0번째, 1번째 칼럼의 0~2행 추출
    ```

### 2.2.4 금융 시계열 데이터 분석에 유용한 pandas 함수
--------------------------------

1. shift()
    * 서로 다른 시간대의 데이터 변화율을 계산할 때 데이터 시점을 지연(lagging)시키는 기법으로 해당 함수를 사용함
    * index 변화 없이 데이터만 전/후로 이동
    EX) 일별 시세 데이터에 대해 당일 거래량이 전날 거래량보다 증가한 경우를 찾을 때 아래와 같이 할 수 있다.
    ```python
    from pykrx import stock
    df = stock.get_market_ohlcv_by_date("20190101", "20190630", "005930")
    df["전날거래량"] = df["거래량"].shift(1)
    cond = df["거래량"] > df["전날거래량"]
    df[cond]
    ```
> shift 함수 인자가 양수일 경우 아래로, 음수일 경우 위로 shift 된다.  
> axis = '0'(row) 가 기본값이며, '1'(column) 일 경우 데이터가 좌우로 이동한다.  

<br/>

2. pct_change()
    * 현재 값과 이전 요소 값의 변화율 (수익률 계산 시 유용)
    <div style="background-color:#FBF8EF; padding:10px;">
        <span style="font-weight:bold;">※ 수익률</span>
        <p>
            &emsp;삼성전자의 어제 종가가 10,000원이었고 오늘은 12,000 원이라고 할 때 하루 동안의 수익률 : (12,000 - 10,000) / 10,000
        </p>
    </div>
    
    ```python
    import pandas as pd
    data = {'삼성전자': [52200, 52300, 52900, 52000, 51700], 
               'LG전자': [68200, 67800, 68800, 67500, 66300]}
    df = pd.DataFrame(data=data) # 삼성전자와 LG전자의 5일 종가 데이터
    df.pct_change() # 삼성전자와 LG전자의 일별 수익률 계산
    df.pct_change() * 100 # 백분율로 나타내기
    df.pct_change(periods=2) * 100 # 2일 전 종가 대비 수익률 계산
    ```  
    
3. diff()
    * 현재 값과 이전 요소 값의 변화량 (두 값의 차)  

<br/>

4. rolling()
    * 일정 구간 데이터(window)들의 mean, min, max 등을 계산
    * 이동 평균선, 지수 이동 평균, 볼린저 밴드 계산 시 응용
    
    <div style="background-color:#FBF8EF; padding:10px;">
        <span style="font-weight:bold;">※ 이동 평균선</span>
        <p>
            &emsp;일정 기간(5일, 10일 등) 단위의 주가 이동 평균치로, 가장 기본적인 smoothing 기법(노이즈 보정)이다.
        </p><br/>
        <span style="font-weight:bold;">※ 단순 이동 평균(SMA)</span>
        <p>
            &emsp;N일간의 종가 평균 <br/>
            &emsp;N일 단순 이동평균은 N일 동안 균등하게 주식 수를 매수/매도하였다고 가정하였을 때의 평균 가격에 해당
        </p><br/>
        <span style="font-weight:bold;">※ 지수 이동 평균(EMA)</span>
        <p>
            &emsp;최근 주가에 가중치를 두어 계산 <br/>
            &emsp;모든 과거의 데이터를 이용하며, N이 커질수록 최근 주가의 가중치가 낮아진다(가중치 : 2 / (1 + N)).
        </p><br/>
        <span style="font-weight:bold;">※ 볼린저 밴드</span>
        <p>
            &emsp;N기간 동안의 이동평균(MA)과 K배 표준변차 상/하위 밴드로 구성됨 <br/>
            &emsp;가격 변동 추세에 대한 보조지표로, 상방을 이탈한 경우 과매수 상태, 하방을 이탈한 경우 과매도 상태로 본다.
        </p><br/>
    </div>
 
    ```python
    df['MA'] = df['C1'].rolling(window=5).mean() # C1 칼럼의 5일 단위 이동 평균 계산
    ```

5. resample()
    * 시간 간격을 재조정하는데 사용
    * up-sampling
        * 샘플의 빈도수 증가 (EX: 분 단위 → 초 단위)
        * 보간법(기존에 알고 있는 데이터를 활용하여 미지의 값을 찾아내는 방법) 사용
    * down-sampling
        * 샘플의 빈도수 감소 (EX: 초 단위 → 분 단위)
        * 집계(군집으로 묶어 조합값 계산) 사용
> 금융에서 시계열 데이터를 다룰 때는 down-sampling 을 많이 사용한다.
```python
import pandas as pd
i = pd.date_range(start = '2019-01-01', end = '2019-10-01', freq = 'B') # 일 단위로 영업일만 추출
s = pd.Series(range(len(i)), index=i) # 날짜를 index로 0 ~ 195 까지의 증가값을 가지는 Series 생성
s.resample(rule = 'M').sum() # 월말 일자를 기준으로 월별 합계를 구한다
s.resample(rule = 'MS').first() # 월초 일자의 값을 구한다
s.resample(rule = 'M').last() # 월말 일자의 값을 구한다
```
> rolling() 함수는 시간 기반 window 작업을 수행하고 resample() 함수는 주기 기반 window 작업을 수행한다.  
<br/>
6. 스스로 공부하기
    * dataframe 합치기 : concat, append, merge
    * 유일한 값 추출 : unique
    * 빠른 함수 연산 : map, apply, group by, join, drop, del, reset_index, rename, ...    

## 2.3 금융 데이터 분석을 위한 오픈API 활용
----

* 금융 및 주가 데이터 가져오기
    * 데이터 구매
        * 정제된 데이터 즉각 확보 가능
        * 서비스 구독 비용과 데이터 활용으로 인한 이득을 따져봐야 함 
    * 금융 웹 페이지 크롤링
        * 웹페이지의 데이터를 가져오는 것으로, 증권사 DB에 있는 데이터를 가져오는 것과 차이가 있음
        * 단시간에 많은 데이터를 요청할 경우 서버 접근이 차단될 수 있음
    * 금융 데이터 제공 오픈 API 사용
        * python 등의 환경에 라이브러리를 설치해 사용
        * 내부적으로는 데이터 서버에서 크롤링해 가져오는 것이기 때문에 과다 사용 시 접근이 차단될 수 있음
        * 접근할 수 있는 데이터가 제한됨
        * 초기에 간단한 작업을 하거나 아이디어를 검증할 때 가장 편하게 사용할 수 있는 방법
    * 증권사 API 이용
        * 직접 프로그래밍해 원하는 데이터를 가져올 수 있음
        * 대표적으로 아래의 API들이 있다.  
    
            구분              | 키움증권 | 대신증권(크레온) | 이베스트투자증권(Xing)
        :-----            |:-----    |:-----            |:-----
        환경              | Windows | Windows | Windows
        API 연결 방식     |	OCX | COM | COM, DLL
        API 사용성        | 보통 | 좋음 | 보통
        요청 제한         | 5/s | 4/s | 10/s
        자동로그인 보안성 | 좋음 | 보통 | 보통
        ※ 출처 : 퀀티랩 블로그|(http://blog.quantylab.com/htsapi.html),| "증권사 API 장단점 비교",| 2018-11-21

### 2.3.2 FinanceDataReader 사용법
---------

* 한국 주식 가격, 미국 주식 가격, 지수, 환율, 암호 화폐 가격, 종목 리스트 등의 데이터를 제공 (웹페이지 크롤링)
* 매뉴얼이 한국어로 설명되어 있어 이해하기 쉽다.
    * https://financedata.github.io/posts/finance-data-reader-users-guide.html
* 다음 명령어를 입력하여 라이브러리를 설치한다.
```python
pip install -U finance-datareader
```
* 라이브러리 사용해보기
```python
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX') # 한국 거래소에 상장된 종목 리스트 가져오기
df_GS2018_price = fdr.DataReader('001250', '2018') # GS글로벌, 2018년 가격 데이터 (단축 코드 사용)
df_GS2018_price['Close'].plot() # 종가 데이터로 그래프 그리기
df_AAPL2017_price = fdr.DataReader('AAPL', '2017') # 애플(AAPL), 2017년 가격 데이터 (ticker 명 사용)
df_bitCoin = fdr.DataReader('BTC/KRW', '2016') # 비트코인 원화가격(빗썸), 2016년 ~ 현재
```
> 미국 이외의 해외 주식 종목을 찾으려면 Investing.com 을 이용한다. (전 세계 대상 금융 데이터 조회 사이트)